In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt 
import matplotlib
from bokeh.layouts import column
from bokeh.models import CustomJS, ColumnDataSource, Slider, HoverTool
from bokeh.plotting import Figure, output_file, show,output_notebook
output_notebook()

Loading BokehJS ...

In [0]:
dataframe_2015=pd.read_csv("2015.csv")
dataframe_2015['Year']=2015
dataframe_2016=pd.read_csv("2016.csv")
dataframe_2016['Year']=2016

#dataframe_2015.set_index('Year',inplace=True)
#dataframe_2016.set_index('Year',inplace=True)


In [0]:
data_f=pd.concat([dataframe_2015[1:100],dataframe_2016[1:100]],axis=0)

data_f = data_f.sample(frac=1)


In [0]:
data_f.head()

,Country,Dystopia Residual,Economy (GDP per Capita),Family,Freedom,Generosity,Happiness Rank,Happiness Score,Health (Life Expectancy),Lower Confidence Interval,Region,Standard Error,Trust (Government Corruption),Upper Confidence Interval,Year
20,United Kingdom,1.96994,1.26637,1.28548,0.59625,0.51912,21,6.867,0.90943,NaN,Western Europe,0.01866,0.32067,NaN,2015
34,Saudi Arabia,2.43872,1.39541,1.08393,0.31048,0.13706,35,6.411,0.72025,NaN,Middle East and Northern Africa,0.04633,0.32524,NaN,2015
37,Algeria,3.40904,1.05266,0.83309,0.21006,0.07044,38,6.355,0.61804,6.227,Middle East and Northern Africa,NaN,0.16157,6.483,2016
6,Netherlands,2.70749,1.46468,1.02912,0.55211,0.47416,7,7.339,0.81231,7.284,Western Europe,NaN,0.29927,7.394,2016
45,El Salvador,3.22134,0.87370,0.80975,0.37269,0.08877,46,6.068,0.59600,5.967,Latin America and Caribbean,NaN,0.10613,6.169,2016


In [4]:
data_f.columns = data_f.columns.str.replace('\s+', '_') 
data_f.columns = data_f.columns.str.replace('(', '') 
data_f.columns = data_f.columns.str.replace(')', '') 
data_f.columns
data=data_f
data.drop('Dystopia_Residual',axis=1,inplace=True)
data.head(5)

,Country,Economy_GDP_per_Capita,Family,Freedom,Generosity,Happiness_Rank,Happiness_Score,Health_Life_Expectancy,Lower_Confidence_Interval,Region,Standard_Error,Trust_Government_Corruption,Upper_Confidence_Interval,Year
25,Germany,1.32792,1.29937,0.61477,0.28214,26,6.750,0.89186,NaN,Western Europe,0.01848,0.21843,NaN,2015
89,Morocco,0.84058,0.38595,0.25646,0.04053,90,5.151,0.59471,5.058,Middle East and Northern Africa,NaN,0.08404,5.244,2016
98,Laos,0.59066,0.73803,0.59591,0.42192,99,4.876,0.54909,NaN,Southeastern Asia,0.06698,0.24249,NaN,2015
56,Poland,1.24585,1.04685,0.45190,0.14443,57,5.835,0.69058,5.749,Central and Eastern Europe,NaN,0.05500,5.921,2016
55,Lithuania,1.14723,1.25745,0.21342,0.02641,56,5.833,0.73128,NaN,Central and Eastern Europe,0.03843,0.01031,NaN,2015


In [0]:
source=ColumnDataSource(dict(
x=data[data['Year']==2015]['Happiness_Rank'],
y=data[data['Year']==2015]['Economy_GDP_per_Capita'],
country=data[data['Year']==2015]['Country'],
region=data[data['Year']==2015]['Region'],
))

plot = Figure(plot_width=900, plot_height=700,
              tools=[HoverTool(tooltips='@country',show_arrow=False)],
              x_axis_label='Happiness Rank',y_axis_label='Economy GDP per Capita')


regions_list = (data.Region.unique()).tolist()

# Import CategoricalColorMapper from bokeh.models 
# and the Spectral6 palette from bokeh.palettes
from bokeh.models import CategoricalColorMapper
from bokeh.palettes import Spectral10

# Make a color mapper: color_mapper
color_mapper = CategoricalColorMapper(factors=regions_list, 
                                      palette=Spectral10)

# Add the color mapper to the circle glyph
plot.circle(x='x', y='y', fill_alpha=5, source=source,
            color=dict(field='region',transform=color_mapper), 
            legend='region')
plot.legend.location = 'bottom_right'

In [6]:
from bokeh.layouts import column
from bokeh.models import CustomJS, ColumnDataSource, Slider
from bokeh.plotting import Figure, output_file, show

output_file("js_on_change.html")


callback = CustomJS(args=dict(source=source), code="""
    var data = source.data;
    var f = cb_obj.value
    var x = data[data['Year']==f]['Happiness_Rank']
    var y = data[data['Year']==f]['Economy_GDP_per_Capita']
    var country = data[data['Year']==f]['Country']
    var region = data[data['Year']==f]['Region']
    source.change.emit();
""")

slider = Slider(start=2015, end=2016, value=2015, step=1, title="power")
slider.js_on_change('value', callback)

layout = column(slider, plot)
output_notebook()
show(layout)

Loading BokehJS ...